In [1]:
import numpy
import pandas as pd
from sklearn import metrics, datasets, manifold
# import matplotlib.pyplot as plt
import tkinter as tk
import tkinter.filedialog
import tkinter.messagebox
import os
import datetime
# from base64 import b64decode
# from PIL import ImageTk


def center_window(root, width, height):
    screenwidth = root.winfo_screenwidth()
    screenheight = root.winfo_screenheight()
    size = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
    # print(size)
    root.geometry(size)
    root.update()
    # print(root.winfo_x())

def file_select():
    filename = tk.filedialog.askopenfilename()
    if filename != "":
        string_filename = ""
        string_filename = str(filename)
        lb.config(text = "您选择的文件是："+string_filename)
    else:
        lb.config(text = "您没有选择任何文件");
    # print("file_select")
    return filename

def data_prepare(filename):
    sheets_data = pd.read_excel(filename,sheet_name = None)
    sheetnames_list = list(sheets_data.keys())
    sheets_data_prepared = {}
    sheets_crowd_names = {}
    sheets_crowd_count = {}
    for sheetname in sheetnames_list:
        df_temp = sheets_data[sheetname]
        # print(df_temp)
        if df_temp.columns[1] != "人群量":
            df_temp["Unnamed: 0"].fillna(method='ffill', inplace=True)
            df_temp.insert(2,"标签名_值",None)
            df_temp["标签名_值"] = df_temp["Unnamed: 0"].astype(str) + "_" + df_temp["Unnamed: 1"].astype(str)
            df_temp.drop(["Unnamed: 0","Unnamed: 1"] ,inplace = True,axis = 1)
            df_temp.set_index(["标签名_值"],inplace=True)
            df_temp.index.name = None
            df_temp = df_temp.T
            df_temp.insert(0,"人群量",df_temp.index)
            df_temp["人群量"].replace(r'(.*)/',"",inplace = True,regex = True)
            df_temp.insert(0,"人群名称",df_temp.index)
            df_temp["人群名称"].replace(r'/(.*)',"",inplace = True,regex = True)
            df_temp.set_index(["人群名称"],inplace=True)
            df_temp.index.name = None
            df_temp.fillna(value=0, inplace=True)
        else:
            df_temp.set_index(["Unnamed: 0"],inplace=True)
            df_temp.index.name = None
        df_temp['人群量'] = pd.to_numeric(df_temp['人群量'])
        for each_column in df_temp.columns[1:]:
            df_temp[each_column] = df_temp[each_column].astype(float)
        sheets_crowd_count[sheetname] = df_temp.iloc[:,:1].values
        sheets_data_prepared[sheetname] = df_temp.iloc[:,1:].values
        sheets_crowd_names[sheetname] = list(df_temp.index)
    # print("data_prepare")
    return sheetnames_list,sheets_data_prepared,sheets_crowd_names,sheets_crowd_count
    # print(sheets_data_prepared)
    # print(sheets_crowd_names)

def loop_io():
    filename = file_select()
    sheetnames_list,sheets_data_prepared,sheets_crowd_names,sheets_crowd_count = data_prepare(filename)
    sheets_output = {}
    for sheetname in sheetnames_list:
        each_sheet_output = MDS(sheets_data_prepared[sheetname],sheets_crowd_names[sheetname],sheets_crowd_count[sheetname])
        sheets_output[sheetname] = each_sheet_output
    current_filepath = os.getcwd()
    time_stamp_str = str(datetime.datetime.now()).split(".")[0].replace(" ","").replace("-","").replace(":","")
    file_to_write = current_filepath + "\MDS_result_" + time_stamp_str + ".xlsx"
    # print("loop_io_before")
    with pd.ExcelWriter(file_to_write) as writer:
        for sheetname in sheetnames_list:
            sheets_output[sheetname].to_excel(writer,index=True,sheet_name = sheetname)
            # print(sheets_output[sheetname])
    # print(file_to_write)
    # print("loop_io")
    tk.messagebox.showinfo('提示',"MDS处理后文件保存在：" + "\n" + file_to_write + "\n" + "可继续选择其他文件或者退出程序")
    # lb.config(text = "MDS处理后文件保存在：" + file_to_write);
    # return file_to_write


def calculate_distance(x, y):
    d = numpy.sqrt(numpy.sum((x - y) ** 2))
    return d

# 计算矩阵各行之间的欧式距离；x矩阵的第i行与y矩阵的第0-j行继续欧式距离计算，构成新矩阵第i行[i0、i1...ij]
def calculate_distance_matrix(x, y):
    d = metrics.pairwise_distances(x, y)
    return d

def cal_B(D):
    (n1, n2) = D.shape
    DD = numpy.square(D)                    # 矩阵D 所有元素平方
    Di = numpy.sum(DD, axis=1) / n1         # 计算dist(i.)^2
    Dj = numpy.sum(DD, axis=0) / n1         # 计算dist(.j)^2
    Dij = numpy.sum(DD) / (n1 ** 2)         # 计算dist(ij)^2
    B = numpy.zeros((n1, n1))
    for i in range(n1):
        for j in range(n2):
            B[i, j] = (Dij + DD[i, j] - Di[i] - Dj[j]) / (-2)   # 计算b(ij)
    return B


def MDS(data,crowd_name,crowd_count,n=2):
    D = calculate_distance_matrix(data, data)
    # print(D)
    B = cal_B(D)
    Be, Bv = numpy.linalg.eigh(B)             # Be矩|B的特征值，Bv归一化的特征向量
    Be_sort = numpy.argsort(-Be)
    Be = Be[Be_sort]                          # 特征值从大到小排序
    Bv = Bv[:, Be_sort]                       # 归一化特征向量
    Bez = numpy.diag(Be[0:n])                 # 前n个特征值对角矩阵
    Bvz = Bv[:, 0:n]                          # 前n个归一化特征向量
    Z = numpy.dot(numpy.sqrt(Bez), Bvz.T).T
    m,n = D.shape
    # f = open("D:/Desktop/2.txt", "r", encoding='utf-8')
    # line = f.readline()  # 读取第一行
    # txt_tables = []
    # while line:
    #     # txt_data = eval(line) # 可将字符串变为元组
    #     txt_tables.append(line)  # 列表增加
    #     line = f.readline()  # 读取下一行
    # label =  txt_tables
    # label = crowd_feature.index.to_list()

    # plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    # for i in range(m):
    #     plt.scatter(Z[i, 0], Z[i, 1], c="r")
    #     plt.annotate(label[i], xy=(Z[i, 0], Z[i, 1]), xytext=(Z[i, 0], Z[i, 1]))
    # plt.show()

    # print(Z)
    sheet_output = pd.DataFrame(Z)
    sheet_output.index = crowd_name
    sheet_output.columns = ["x","y"]
    sheet_output.insert(len(sheet_output.columns),"count",crowd_count)
    #     mySeries.insert(len(mySeries.columns),"count",crowd_data["总量级"].values)
    #     mySeries.to_excel('output_20211022.xlsx', encoding='utf-8', index=True)
    # numpy.savetxt('C:/Users/huo/Desktop/output.txt', Z, delimiter=',')
    # print("MDS")
    return sheet_output

if __name__ == '__main__':
    root = tk.Tk()
    root.title('MDS')
    # root.geometry('500x300+960+540')
    center_window(root, 500, 200)
    # root.iconbitmap("haha.ico")
    lb = tk.Label(root,text = '')
    lb.pack()
    btn = tk.Button(root,text="请选择文件",command = loop_io)
    btn.pack()
    # print(btn.winfo_width())
    # print(btn.winfo_height())
    btn.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
    root.mainloop()
# data = numpy.loadtxt('D:/Desktop/1.txt', delimiter='\t')
# data = numpy.mat(data)
# data = crowd_feature.values
# Z = MDS(data)

# self.init_window_name.iconbitmap(default = "D:\Desktop\bubble\haha.ico")
# ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID("MY_GUI")

# import base64
#
# with open("haha.ico", 'rb') as open_icon:
#     b64str = base64.b64encode(open_icon.read())
#     print(b64str)